In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/Sapienza/DL

/content/drive/MyDrive/Sapienza/DL


In [3]:
import pickle
import pandas as pd
import numpy as np
import json
import itertools
import gensim
path="SEMEVAL-2021-task6-corpus/data/"


In [4]:
with open('training_task1_labels.pkl', 'rb') as handle:
    train_labs = pickle.load(handle)

with open('original_training_task1_features.pkl', 'rb') as handle:
    train_feats = pickle.load(handle)

with open('training_task1_all_labels.pkl', 'rb') as handle:
    all_labels = pickle.load(handle)

with open('dev_task1_labels.pkl', 'rb') as handle:
    dev_labs = pickle.load(handle)

with open('original_dev_task1_features.pkl', 'rb') as handle:
    dev_feats = pickle.load(handle)

with open('test_task1_labels.pkl', 'rb') as handle:
    test_labs = pickle.load(handle)

with open('original_test_task1_features.pkl', 'rb') as handle:
    test_feats = pickle.load(handle)


In [14]:
train_feats[:3]

['THERE ARE ONLY TWO GENDERS FEMALE  MALE',
 'This is not an accident!',
 'SO BERNIE BROS HAVENT COMMITTED VIOLENCE EH? POWER COMES FROM THE BARREL OF A GUN, COMRADES. WHAT ABOUT THE ONE WHO SHOT CONGRESSMAN SCALISE OR THE DAYTON OHIO MASS SHOOTER?']

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(train_feats)
word_index = tokenizer.word_index

In [6]:
!pip install spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.1 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=19816d913ceced124a7057846af9fb2fae2ef1e815656dd2901784cd4d7f62a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-lptcq21e/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [6]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [7]:
word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'you': 4,
 'and': 5,
 'is': 6,
 'of': 7,
 'for': 8,
 'in': 9,
 'i': 10,
 'that': 11,
 'this': 12,
 'it': 13,
 'trump': 14,
 'on': 15,
 'we': 16,
 'are': 17,
 'your': 18,
 'not': 19,
 'if': 20,
 'have': 21,
 'be': 22,
 'he': 23,
 'they': 24,
 'no': 25,
 'who': 26,
 'when': 27,
 'me': 28,
 'by': 29,
 'with': 30,
 'was': 31,
 'all': 32,
 'what': 33,
 'people': 34,
 'dont': 35,
 'my': 36,
 'can': 37,
 'do': 38,
 'its': 39,
 'will': 40,
 'president': 41,
 'his': 42,
 'at': 43,
 'america': 44,
 'im': 45,
 'but': 46,
 'just': 47,
 'from': 48,
 'how': 49,
 'biden': 50,
 'like': 51,
 'get': 52,
 'their': 53,
 'has': 54,
 'about': 55,
 'were': 56,
 'up': 57,
 'so': 58,
 'joe': 59,
 'one': 60,
 'think': 61,
 'an': 62,
 'only': 63,
 'democrats': 64,
 'out': 65,
 'or': 66,
 'because': 67,
 '•': 68,
 'want': 69,
 '2020': 70,
 'now': 71,
 'man': 72,
 'there': 73,
 'them': 74,
 'know': 75,
 'our': 76,
 'being': 77,
 'would': 78,
 'black': 79,
 'did': 80,
 'make': 81,
 'y

In [8]:
text_embedding = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
  text_embedding[i] = nlp(word).vector

In [9]:
text_embedding.shape

(3203, 300)

In [10]:
MAX_SEQUENCE_LENGTH=50

In [11]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

model = Sequential()
model.add(Embedding(input_dim=text_embedding.shape[0], output_dim=text_embedding.shape[1], weights=[text_embedding], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(64, kernel_size=5,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64,dropout=0.5, recurrent_dropout=0.2,return_sequences=True)))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(32, kernel_size=5,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(32,dropout=0.2, recurrent_dropout=0.5,return_sequences=True)))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(16, kernel_size=5,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(16,dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512,activation='relu'))
model.add(Dense(23,activation='sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 300)           960900    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 300)          0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            96064     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 50, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 25, 64)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 25, 128)          6

In [12]:
X_train=train_feats
X_test=dev_feats
Y_train=train_labs
Y_test=dev_labs

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

model.fit(
 pad_sequences(
   tokenizer.texts_to_sequences(X_train), maxlen = MAX_SEQUENCE_LENGTH
 ),
 Y_train, batch_size = 64, epochs = 20,
 validation_data = (
   pad_sequences(
     tokenizer.texts_to_sequences(X_test), maxlen = MAX_SEQUENCE_LENGTH
   ),
   Y_test
 ),
 shuffle = True
)

Epoch 1/20
11/11 [==============================] - 19s 331ms/step - loss: 5.7677 - accuracy: 0.1061 - val_loss: 5.7403 - val_accuracy: 0.0476
Epoch 2/20
11/11 [==============================] - 2s 194ms/step - loss: 4.7868 - accuracy: 0.1032 - val_loss: 5.9285 - val_accuracy: 0.0476
Epoch 3/20
11/11 [==============================] - 2s 183ms/step - loss: 5.2927 - accuracy: 0.1032 - val_loss: 7.0756 - val_accuracy: 0.0476
Epoch 4/20
11/11 [==============================] - 2s 183ms/step - loss: 6.6609 - accuracy: 0.1032 - val_loss: 9.1215 - val_accuracy: 0.0476
Epoch 5/20
11/11 [==============================] - 2s 186ms/step - loss: 8.3049 - accuracy: 0.1032 - val_loss: 11.2155 - val_accuracy: 0.0476
Epoch 6/20
11/11 [==============================] - 2s 191ms/step - loss: 10.0554 - accuracy: 0.1032 - val_loss: 13.0847 - val_accuracy: 0.0476
Epoch 7/20
11/11 [==============================] - 2s 184ms/step - loss: 11.4776 - accuracy: 0.1090 - val_loss: 14.8452 - val_accuracy: 0.2063


KeyboardInterrupt: ignored